In [1]:
# Install packages
!pip install langchain langchain-core langchain-community pydantic langchain-experimental -q #duckduckgo_search (not installed as issuing lot of warnings )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


**Built-in Tool - DuckDuckGo Search**

In [2]:
"""
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

search_tool.invoke("latest news in india?")"""

'\nfrom langchain_community.tools import DuckDuckGoSearchRun\n\nsearch_tool = DuckDuckGoSearchRun()\n\nsearch_tool.invoke("latest news in india?")'

In [3]:
"""
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)"""

'\nprint(search_tool.name)\nprint(search_tool.description)\nprint(search_tool.args)'

**Built-in Tool - Shell Tool**

In [4]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


**Custom Tools**

In [5]:
from langchain_community.tools import tool

In [6]:
# Step 1: Create function

def multiply(a,b):
  """Mutiply two numbers"""
  return a*b

In [7]:
# Step 2: Add type hints
def multiply (a:int, b:int) -> int:
  """Mutiply two numbers"""
  return a*b

In [8]:
# Step 3: Add tool decorator
@tool
def multiply(a:int, b:int) -> int:
  """Mutiply two numbers"""
  return a*b

In [9]:
results = multiply.invoke({"a":50, "b":50})
print(results)

2500


In [10]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Mutiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
print(multiply.args_schema.model_json_schema())

{'description': 'Mutiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


**Method 2 - Using StructuredTool**

In [12]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [13]:
 class MultiplyInput(BaseModel):
  a: int = Field(required = True, description = "The first number to add")
  b: int = Field(required = True, description = "The second number to add")

In [14]:
def multiply_func(a:int, b:int)-> int:
  return a*b

In [15]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",
    description = "Multiply two numbers",
    args_schema = MultiplyInput
)

In [16]:
result =multiply_tool.invoke({'a':450, "b":297})

In [17]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

133650
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


**Method 3 - Using BaseTool Class**

In [18]:
from langchain.tools import BaseTool
from typing import Type

In [19]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
  a: int = Field(required=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

In [20]:
class MultiplyTool(BaseTool):
  name: str = "multiply"
  description: str = "Multiply two numbers"
  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a:int, b:int) ->int:
    return a * b

In [21]:
multiply_tool = MultiplyTool()

In [22]:
result = multiply_tool.invoke({'a':450, "b":297})

In [23]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

133650
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


**Toolkit**

In [24]:
from langchain_core.tools import tool

#Custom tools

@tool
def add (a:int, b:int) -> int:
  """Add two numbers"""
  return a + b

@tool
def multiply (a:int, b:int) -> int:
  """Multiply two numbers"""
  return a * b


In [25]:
class MathToolKit:
  def get_tools(self):
    return[add, multiply]

In [26]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name,"=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
